In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate

from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
chunk_size = 100000  # Adjust as needed
chunks = pd.read_csv('E:/Phat/data_electricity.csv', chunksize=chunk_size, low_memory=False, na_values='\\N')

data = pd.concat([chunk for chunk in chunks])

In [4]:
for col in data.select_dtypes(include=['float64']).columns:
    data[col] = data[col].astype(np.float32)

for col in data.select_dtypes(include=['int64']).columns:
    data[col] = data[col].astype(np.int32)

In [5]:
data.head(5)

,ma_dviqly,id_chiso,id_bcs,ma_dvictren,ma_ddo,bcs_chiso,hs_nhan,so_cto_chiso,ky,thang,nam,chiso_cu,chiso_moi,san_luong,sluong_ttiep,sluong_trphu,loai_chiso,thd_le,ma_ttcto,ngay_dky,ngay_cky,sluong_1,sluong_2,sluong_3,ngay_tao_chiso,nguoi_tao_chiso,ngay_sua_chiso,nguoi_sua_chiso,ma_cnang_chiso,so_cot,so_hop,ma_tram,ngay_hluc_vitri_ddo,ngay_hhluc_vitri_ddo,ma_cto,so_cto_hso_cto,ngay_bdong,ngay_kdinh,ma_bdong,dong_dien,dien_ap,vh_cong,ma_cloai,so_pha,id_khang,ten_khang,ma_khang,ngay_hluc_khang,ngay_hhluc_khang,dia_chi,ngay_hluc_diem_do,ngay_hhluc_diemdo,id_ddo,kimua_cspk,csuat,so_cto_csuat_ddo,thang_csuat_ddo,loai_giatri,gia_tri,ngay_hluc,ngay_hhluc,fraud
0,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,11,PMAX,0.0,2000-12-23,NaN,0
1,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,12,PMAX,0.0,2000-12-23,NaN,0
2,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,1,PMAX,0.0,2000-12-23,NaN,0
3,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,2,PMAX,0.0,2000-12-23,NaN,0
4,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,3,PMAX,0.0,2000-12-23,NaN,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22178305 entries, 0 to 22178304
Data columns (total 62 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ma_dviqly             object 
 1   id_chiso              int32  
 2   id_bcs                int32  
 3   ma_dvictren           object 
 4   ma_ddo                object 
 5   bcs_chiso             object 
 6   hs_nhan               float32
 7   so_cto_chiso          int32  
 8   ky                    int32  
 9   thang                 int32  
 10  nam                   int32  
 11  chiso_cu              float32
 12  chiso_moi             float32
 13  san_luong             int32  
 14  sluong_ttiep          int32  
 15  sluong_trphu          int32  
 16  loai_chiso            object 
 17  thd_le                int32  
 18  ma_ttcto              object 
 19  ngay_dky              object 
 20  ngay_cky              object 
 21  sluong_1              int32  
 22  sluong_2              int32  
 23  sluon

In [7]:
date_cols = ['ngay_dky', 'ngay_cky', 'ngay_hluc_vitri_ddo', 'ngay_hhluc_vitri_ddo', 'ngay_bdong', 'ngay_kdinh',
    'ngay_hluc_khang', 'ngay_hhluc_khang', 'ngay_hluc_diem_do', 'ngay_hhluc_diemdo',
    'ngay_hluc', 'ngay_hhluc'
]
datetime_cols = ['ngay_tao_chiso', 'ngay_sua_chiso']

def process_date_pandas(df_pandas):
    for col in date_cols:
        df_pandas[col] = pd.to_datetime(df_pandas[col],  format="%Y-%m-%d", errors="coerce")

        for attribute in ['year', 'month', 'day']:
            tqdm.pandas(desc=f"Extracting {attribute} of {col}")
            df_pandas[f"{col}_{attribute}"] = df_pandas[col].progress_apply(lambda x: getattr(x, attribute) if pd.notnull(x) else np.nan)
    
    return df_pandas

def process_datetime_pandas(df_pandas):
    for col in datetime_cols:
        df_pandas[col] = pd.to_datetime(df_pandas[col],  format="%Y-%m-%d %H:%M:%S", errors="coerce")

        for attribute in ['year', 'month', 'day']:
            tqdm.pandas(desc=f"Extracting {attribute} of {col}")
            df_pandas[f"{col}_{attribute}"] = df_pandas[col].progress_apply(lambda x: getattr(x, attribute) if pd.notnull(x) else np.nan)
    
    return df_pandas

data = process_datetime_pandas(data) 
data = process_date_pandas(data)    

Extracting day of ngay_hhluc: 100%|██████████| 22178305/22178305 [01:54<00:00, 194430.27it/s]


In [8]:
data.isna().sum()

ma_dviqly                            0
id_chiso                             0
id_bcs                               0
ma_dvictren                          0
ma_ddo                               0
bcs_chiso                            0
hs_nhan                              0
so_cto_chiso                         0
ky                                   0
thang                                0
nam                                  0
chiso_cu                             0
chiso_moi                            0
san_luong                            0
sluong_ttiep                         0
sluong_trphu                         0
loai_chiso                           0
thd_le                               0
ma_ttcto                      21946080
ngay_dky                             0
ngay_cky                             0
sluong_1                             0
sluong_2                             0
sluong_3                             0
ngay_tao_chiso                       0
nguoi_tao_chiso          

In [9]:
data.head(5)

,ma_dviqly,id_chiso,id_bcs,ma_dvictren,ma_ddo,bcs_chiso,hs_nhan,so_cto_chiso,ky,thang,nam,chiso_cu,chiso_moi,san_luong,sluong_ttiep,sluong_trphu,loai_chiso,thd_le,ma_ttcto,ngay_dky,ngay_cky,sluong_1,sluong_2,sluong_3,ngay_tao_chiso,nguoi_tao_chiso,ngay_sua_chiso,nguoi_sua_chiso,ma_cnang_chiso,so_cot,so_hop,ma_tram,ngay_hluc_vitri_ddo,ngay_hhluc_vitri_ddo,ma_cto,so_cto_hso_cto,ngay_bdong,ngay_kdinh,ma_bdong,dong_dien,dien_ap,vh_cong,ma_cloai,so_pha,id_khang,ten_khang,ma_khang,ngay_hluc_khang,ngay_hhluc_khang,dia_chi,ngay_hluc_diem_do,ngay_hhluc_diemdo,id_ddo,kimua_cspk,csuat,so_cto_csuat_ddo,thang_csuat_ddo,loai_giatri,gia_tri,ngay_hluc,ngay_hhluc,fraud,ngay_tao_chiso_year,ngay_tao_chiso_month,ngay_tao_chiso_day,ngay_sua_chiso_year,ngay_sua_chiso_month,ngay_sua_chiso_day,ngay_dky_year,ngay_dky_month,ngay_dky_day,ngay_cky_year,ngay_cky_month,ngay_cky_day,ngay_hluc_vitri_ddo_year,ngay_hluc_vitri_ddo_month,ngay_hluc_vitri_ddo_day,ngay_hhluc_vitri_ddo_year,ngay_hhluc_vitri_ddo_month,ngay_hhluc_vitri_ddo_day,ngay_bdong_year,ngay_bdong_month,ngay_bdong_day,ngay_kdinh_year,ngay_kdinh_month,ngay_kdinh_day,ngay_hluc_khang_year,ngay_hluc_khang_month,ngay_hluc_khang_day,ngay_hhluc_khang_year,ngay_hhluc_khang_month,ngay_hhluc_khang_day,ngay_hluc_diem_do_year,ngay_hluc_diem_do_month,ngay_hluc_diem_do_day,ngay_hhluc_diemdo_year,ngay_hhluc_diemdo_month,ngay_hhluc_diemdo_day,ngay_hluc_year,ngay_hluc_month,ngay_hluc_day,ngay_hhluc_year,ngay_hhluc_month,ngay_hhluc_day
0,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,11,PMAX,0.0,2000-12-23,NaT,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,NaN,NaN,NaN
1,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,12,PMAX,0.0,2000-12-23,NaT,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,NaN,NaN,NaN
2,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,1,PMAX,0.0,2000-12-23,NaT,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,NaN,NaN,NaN
3,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0,1512016626,1,12,2019,14210.0,14536.0,326,0,0,DDK,0,NaN,2019-11-12,2019-12-11,365,448,431,2019-12-12 08:23:23,THUYNT1,2019-12-13 09:22:24,THUYNT1,56,NaN,70,PD0182224,2007-07-12,2016-06-16,398173938,1512016626,2016-03-09,2016-03-04,B,5(80),220,D1,578,1,3479352,####Đình T###,PD0100T3248,2007-07-12,2016-06-16,####Hàng Đào#,2007-07-12,2016-06-16,5316005,0,0.0,1512016626,2,PMAX,0.0,2000-12-23,NaT,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,NaN,NaN,NaN
4,PD0100,-1659820155,24542382,PD,PD0100T32481,KT,1.0

In [10]:
data.drop(date_cols, axis=1, inplace=True)

In [11]:
cat_missing_value_cols = ['ma_ttcto', 'so_cot', 'so_hop']
date_missing_value_cols = ['ngay_hluc_vitri_ddo_year', 'ngay_hluc_vitri_ddo_month', 'ngay_hluc_vitri_ddo_day',
                      'ngay_hhluc_vitri_ddo_year', 'ngay_hhluc_vitri_ddo_month', 'ngay_hhluc_vitri_ddo_day',
                      'ngay_hhluc_khang_year', 'ngay_hhluc_khang_month', 'ngay_hhluc_khang_day',
                      'ngay_hhluc_diemdo_year', 'ngay_hhluc_diemdo_month', 'ngay_hhluc_diemdo_day',
                      'ngay_hhluc_year', 'ngay_hhluc_month', 'ngay_hhluc_day']
for col in cat_missing_value_cols:
    data[col].fillna("Missing", inplace=True)
for col in date_missing_value_cols:
    data[col].fillna(-1, inplace=True)

In [12]:
numerical_cols = ['id_chiso','id_bcs','hs_nhan','so_cto_chiso',
                  'chiso_cu','chiso_moi','san_luong','sluong_ttiep','sluong_trphu',
                   'thd_le','sluong_1','sluong_2','sluong_3','ma_cto','so_cto_hso_cto',
                    'ma_cloai','so_pha','id_khang','id_ddo','kimua_cspk','csuat','so_cto_csuat_ddo',
                     'thang_csuat_ddo','gia_tri' ]

scaler = StandardScaler()
for col in numerical_cols:
    data[col] = scaler.fit_transform(data[[col]])

In [13]:
cat_cols = ['ma_dviqly','ma_dvictren','ma_ddo','bcs_chiso','loai_chiso',
            'ma_ttcto','nguoi_tao_chiso','nguoi_sua_chiso','ma_cnang_chiso','so_cot','so_hop',
            'ma_tram','ma_bdong','dong_dien','dien_ap','vh_cong','ten_khang',
            'ma_khang','dia_chi','loai_giatri']

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
for col in cat_cols:
    data[col] = oe.fit_transform(data[[col]].astype(str)).astype(int)

In [14]:
data.drop(datetime_cols, axis=1, inplace=True)

In [15]:
data.head()

,ma_dviqly,id_chiso,id_bcs,ma_dvictren,ma_ddo,bcs_chiso,hs_nhan,so_cto_chiso,ky,thang,nam,chiso_cu,chiso_moi,san_luong,sluong_ttiep,sluong_trphu,loai_chiso,thd_le,ma_ttcto,sluong_1,sluong_2,sluong_3,nguoi_tao_chiso,nguoi_sua_chiso,ma_cnang_chiso,so_cot,so_hop,ma_tram,ma_cto,so_cto_hso_cto,ma_bdong,dong_dien,dien_ap,vh_cong,ma_cloai,so_pha,id_khang,ten_khang,ma_khang,dia_chi,id_ddo,kimua_cspk,csuat,so_cto_csuat_ddo,thang_csuat_ddo,loai_giatri,gia_tri,fraud,ngay_tao_chiso_year,ngay_tao_chiso_month,ngay_tao_chiso_day,ngay_sua_chiso_year,ngay_sua_chiso_month,ngay_sua_chiso_day,ngay_dky_year,ngay_dky_month,ngay_dky_day,ngay_cky_year,ngay_cky_month,ngay_cky_day,ngay_hluc_vitri_ddo_year,ngay_hluc_vitri_ddo_month,ngay_hluc_vitri_ddo_day,ngay_hhluc_vitri_ddo_year,ngay_hhluc_vitri_ddo_month,ngay_hhluc_vitri_ddo_day,ngay_bdong_year,ngay_bdong_month,ngay_bdong_day,ngay_kdinh_year,ngay_kdinh_month,ngay_kdinh_day,ngay_hluc_khang_year,ngay_hluc_khang_month,ngay_hluc_khang_day,ngay_hhluc_khang_year,ngay_hhluc_khang_month,ngay_hhluc_khang_day,ngay_hluc_diem_do_year,ngay_hluc_diem_do_month,ngay_hluc_diem_do_day,ngay_hhluc_diemdo_year,ngay_hhluc_diemdo_month,ngay_hhluc_diemdo_day,ngay_hluc_year,ngay_hluc_month,ngay_hluc_day,ngay_hhluc_year,ngay_hhluc_month,ngay_hhluc_day
0,0,0.396935,-1.785552,0,35527,5,-0.171752,1.232408,1,12,2019,-0.12121,-0.126813,-0.259059,0.0,-0.017875,0,0.0,0,-0.283458,-0.293975,-0.33235,5,5,2,9,621,119,0.015463,1.232408,1,8,0,0,-0.759417,-1.235065,-0.363205,20214,34852,8132,-0.402745,-0.352647,-0.199681,1.232408,1.087823,0,-0.377852,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,-1.0,-1.0,-1.0
1,0,0.396935,-1.785552,0,35527,5,-0.171752,1.232408,1,12,2019,-0.12121,-0.126813,-0.259059,0.0,-0.017875,0,0.0,0,-0.283458,-0.293975,-0.33235,5,5,2,9,621,119,0.015463,1.232408,1,8,0,0,-0.759417,-1.235065,-0.363205,20214,34852,8132,-0.402745,-0.352647,-0.199681,1.232408,1.379915,0,-0.377852,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,-1.0,-1.0,-1.0
2,0,0.396935,-1.785552,0,35527,5,-0.171752,1.232408,1,12,2019,-0.12121,-0.126813,-0.259059,0.0,-0.017875,0,0.0,0,-0.283458,-0.293975,-0.33235,5,5,2,9,621,119,0.015463,1.232408,1,8,0,0,-0.759417,-1.235065,-0.363205,20214,34852,8132,-0.402745,-0.352647,-0.199681,1.232408,-1.833106,0,-0.377852,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,-1.0,-1.0,-1.0
3,0,0.396935,-1.785552,0,35527,5,-0.171752,1.232408,1,12,2019,-0.12121,-0.126813,-0.259059,0.0,-0.017875,0,0.0,0,-0.283458,-0.293975,-0.33235,5,5,2,9,621,119,0.015463,1.232408,1,8,0,0,-0.759417,-1.235065,-0.363205,20214,34852,8132,-0.402745,-0.352647,-0.199681,1.232408,-1.541013,0,-0.377852,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,-1.0,-1.0,-1.0
4,0,0.396935,-1.785552,0,35527,5,-0.171752,1.232408,1,12,2019,-0.12121,-0.126813,-0.259059,0.0,-0.017875,0,0.0,0,-0.283458,-0.293975,-0.33235,5,5,2,9,621,119,0.015463,1.232408,1,8,0,0,-0.759417,-1.235065,-0.363205,20214,34852,8132,-0.402745,-0.352647,-0.199681,1.232408,-1.248920,0,-0.377852,0,2019,12,12,2019,12,13,2019,11,12,2019,12,11,2007.0,7.0,12.0,2016.0,6.0,16.0,2016,3,9,2016,3,4,2007,7,12,2016.0,6.0,16.0,2007,7,12,2016.0,6.0,16.0,2000,12,23,-1.0,-1.0,-1.0


In [16]:
for col in data.select_dtypes(include=['float64']).columns:
    data[col] = data[col].astype(np.float32)

for col in data.select_dtypes(include=['int64']).columns:
    data[col] = data[col].astype(np.int32)

In [17]:
data.to_parquet('data_electricity_processing.parquet', index=False)